In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
debashishsau_aslamerican_sign_language_aplhabet_dataset_path = kagglehub.dataset_download('debashishsau/aslamerican-sign-language-aplhabet-dataset')

print('Data source import complete.')


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D,
    Flatten, Dense, Dropout
)
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt


In [ ]:
!ls /kaggle/input


In [ ]:
!ls /kaggle/input/aslamerican-sign-language-aplhabet-dataset


In [ ]:
!ls /kaggle/input/aslamerican-sign-language-aplhabet-dataset/ASL_Alphabet_Dataset


In [ ]:
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices("GPU"))


In [ ]:
IMG_SIZE = 64
BATCH_SIZE = 32
EPOCHS = 30
SEED = 123
NUM_CLASSES = 29


In [ ]:
DATA_DIR = "/kaggle/input/aslamerican-sign-language-aplhabet-dataset/ASL_Alphabet_Dataset/asl_alphabet_train"


In [ ]:
train_data = image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

test_data = image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)


In [ ]:
print("Number of classes:", len(train_data.class_names))
print("Classes:", train_data.class_names)


In [ ]:
print(len(train_data.class_names))
print(train_data.class_names)


In [ ]:
print("Train batches:", train_data.cardinality().numpy())
print("Validation batches:", test_data.cardinality().numpy())


In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_data = train_data.map(lambda x, y: (normalization_layer(x), y))
test_data = test_data.map(lambda x, y: (normalization_layer(x), y))

train_data = train_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_data = test_data.cache().prefetch(buffer_size=tf.data.AUTOTUNE)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    BatchNormalization
)
from tensorflow.keras import regularizers

model = Sequential([
    # Block 1
    Conv2D(
        16, (3,3),
        padding="same",
        activation="relu",
        kernel_regularizer=regularizers.l2(0.001),
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    ),
    BatchNormalization(),
    MaxPooling2D(2),
    Dropout(0.2),

    # Block 2
    Conv2D(
        32, (3,3),
        padding="same",
        activation="relu",
        kernel_regularizer=regularizers.l2(0.001)
    ),
    BatchNormalization(),
    MaxPooling2D(2),
    Dropout(0.25),

    # Block 3
    Conv2D(
        64, (3,3),
        padding="same",
        activation="relu",
        kernel_regularizer=regularizers.l2(0.001)
    ),
    BatchNormalization(),
    MaxPooling2D(2),
    Dropout(0.3),

    # Block 4
    Conv2D(
        128, (3,3),
        padding="same",
        activation="relu",
        kernel_regularizer=regularizers.l2(0.001)
    ),
    BatchNormalization(),
    MaxPooling2D(2),
    Dropout(0.4),

    # Classifier
    Flatten(),
    Dense(
        128,
        activation="relu",
        kernel_regularizer=regularizers.l2(0.001)
    ),
    BatchNormalization(),
    Dropout(0.5),

    Dense(NUM_CLASSES, activation="softmax")
])


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)


In [ ]:
model.summary()


In [ ]:
history = model.fit(
    train_data,
    epochs=EPOCHS,
    validation_data=test_data
)


In [ ]:
model.save("asl_15epoch.keras")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history["accuracy"], label="Train")
plt.plot(history.history["val_accuracy"], label="Validation")
plt.title("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history["loss"], label="Train")
plt.plot(history.history["val_loss"], label="Validation")
plt.title("Loss")
plt.legend()

plt.show()
